# PRISMA Results Visualization

In this notebook, we visualize the output files generated by PRISMA. It is divided into two main sections:

    * Section 1: explores the intrinsic properties saved directly from the simulation.

    * Section 2: shows a synthetic stellar spectrum and computes the flux under the Hα emission line for a selected spaxel.

Make sure to run PRISMA.ipynb beforehand, as it generates the necessary data stored in the spaxels_information/ and out_dirs/ directories.

In [ ]:
import os
import sys
sys.path.append('Scripts/')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from astropy.io import fits
from astropy import constants as const
from astropy import units as u

In [ ]:
#galaxy information
sim_name = "147062" 
galaxy_id = '9110'
snap = 128

#Rotation information
rot_axis = "y"
angle = 0

#Other parameters 
age_threshold = 1e7  #yr  limit for the young pop.
neb_f_esc = 0.1  #i.e., 10% of the photons escape and do not contribute to the nebular emission
neb_f_dust = 0.5   #i.e., 50% of the photons are absorbed by dust within the SF regions


#local path
data_path = "../"

In [ ]:
#required format definitions:
neb_f_esc_str = str(neb_f_esc).replace(".","")  #to remove the dot in the str
neb_f_dust_str = str(neb_f_dust).replace(".","")  #to remove the dot in the str
age_threshold_format = "{:.0e}".format(age_threshold) #to rewrite the age in other format

## Intrinsic properties

In [ ]:
intrin_data = pd.read_csv(f"{data_path}/spaxels_information/{sim_name}/intrinsic_information_spaxels_{sim_name}-{galaxy_id}_fesc{neb_f_esc_str}_fdust{neb_f_dust_str}.csv")

In [ ]:
intrin_data[:]

## Synthetic spectrum example

In [ ]:
def synthetic_spectrum(out_path,ms,star_idx=0):
    """
    This function allows us to visualize the spectra generated for one star.
    Input -> - out_path :  spectra directory
             - ms       :  stellar mass (as an array with the stellar mass of all the star into the spaxels)
    Output -> Wavelength (nm) and Luminosity (in watts)         
    """
    files = !ls $out_path
    spec = [i for i in files if 'best_model' in i] #to get the modeled spectra files
    spec = [f"{out_path}/{i}" for i in spec] ##to get the full name of the spectra directory
    hdul = [fits.open(i) for i in spec] 
    
    wl = [i[1].data['wavelength'] for i in hdul]
    L_norm_tot = [i[1].data['L_lambda_total'] for i in hdul] 
    L_tot = [L_norm_tot[i] * ms[i] for i in range(len(ms))] #L*M
    
    return(wl[star_idx] , L_tot[star_idx])


def F_to_L(Flux , Dist=10,Watt=True):
    """
    Util function for transforming flux to luminosity considering a distance in pc.
    Input -> - Flux : flux in W/m^2
             - Dist : distance in pc 
    Output -> luminosity in Watts (if True) or erg/s
    """
    Dist = Dist* 30856775812799588 #meters 
    
    if Watt == True:
        return Flux * (4*np.pi * Dist * Dist)
    else: #erg/s
        L_W = Flux * (4*np.pi * Dist * Dist) * u.joule/u.s
        L_ergs = L_W.to(u.erg/u.s)#1W = 1e7 erg/s 
        return(L_ergs.value)


def lines_luminosities(line_name, model_path, m_gas, normalize = False):
    """
    With this function we can obtain the flux under the area of specific emission lines.
    Inputs ->   - line_name : name of the emission line (e.g., line.H-alpha,
                                                               line.H-beta,
                                                               line.OIII-500.7', ...)
                - model_path: path to the spectra directory
                - m_gas: mass of the gas clouds (array with the dimensions of the young populations)
                
    Output -> array with the line emission (in Watts(default) or erg/s) per young stellar population into the spaxel
               each spaxel.
    """
    models = fits.open(model_path)
    line_Fnu = models[1].data[line_name]  
    line_L_norm = F_to_L(line_Fnu, Watt = True)  #normalized to 1 M_sun, set Watt = False to get ergs/s
    if normalize == False:
        line_L = np.array([line_L_norm[i] * m_gas[i] for i in range(len(m_gas))]) #line*Mgas
    else: line_L = line_L_norm
    line_L = line_L[line_L != 0] #to remove the zero luminosity (equivalent to the older contribution)
    return line_L 

In [ ]:
spec_data_path = f"{data_path}/out_dirs/{sim_name}/Synthetic_spectra" #CIGALE spectra directory

### Stellar spectrum visualization

In [ ]:
cond = f"fesc{neb_f_esc_str}_fdust{neb_f_dust_str}"

# Selecting one spaxel
spx = np.unique(intrin_data['Spax_id'])[0]
# Stellar mass array
ms = intrin_data[intrin_data["Spax_id"] == spx]['M_stars']   
# Spectra path
spec_path = f"{spec_data_path}/out_{sim_name}-{galaxy_id}_{cond}_{int(spx)}" 

# Synthetic spectrum of one star into the spaxel
wl_star , L_star = synthetic_spectrum(out_path,ms)

In [ ]:
plt.figure(figsize=(7,3))
plt.plot(wl_star,L_star,color="C9",label="total spectrum",lw=2)
plt.xscale("log")
plt.yscale("log")
plt.legend(fontsize=17)
plt.xlabel(r'$\lambda$ (nm)',fontsize=15)
plt.ylabel(r'$L / \lambda$ (W)',fontsize=15)
plt.show()

### Calculation of the flux under the Halpha line

In [ ]:
# Selecting one spaxel
spx = np.unique(intrin_data['Spax_id'])[0]

# Gas mass
mg_cloud = intrin_data[intrin_data["Spax_id"] == spx]['M_gas_cloud']

In [ ]:
model_path = f"{spec_data_path}/out_{sim_name}-{galaxy_id}_{cond}_{int(spx)}"
H_alpha = lines_luminosities('line.H-alpha',
                             f'{model_path}/models-block-0.fits',
                             mg_cloud) 

In [ ]:
print(H_alpha) #array with the Halpha emission per young stellar population into the spaxel